In [0]:
import json
import yaml
import re
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
from datetime import datetime
import os
import uuid

CATALOG = "ds_training_1"
SCHEMA = "thoughtspot_inventory_ak"

TML_VOLUME = "tml_files_ak"
LVDASH_VOLUME = "lvdash_files_ak_out"

TML_INPUT_PATH = f"/Volumes/ds_training_1/thoughtspot_inventory_ak/lvdash_files_ak/liveboard/All_chart_Liveboard.liveboard.tml"
LVDASH_OUTPUT_PATH = f"/Volumes/{CATALOG}/{SCHEMA}/{LVDASH_VOLUME}/"

TRACKER_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_tracker"
SUMMARY_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_summary"
FAILURE_TABLE = f"{CATALOG}.{SCHEMA}.tml_conversion_failures"
MAPPING_TABLE = f"{CATALOG}.{SCHEMA}.tml_dbx_metadata_mapping"

# Configuration tables
CHART_TYPE_MAPPING_TABLE = f"{CATALOG}.{SCHEMA}.chart_type_mappings"
WIDGET_SIZE_CONFIG_TABLE = f"{CATALOG}.{SCHEMA}.widget_size_config"
EXPRESSION_TRANSFORM_TABLE = f"{CATALOG}.{SCHEMA}.expression_transformations"
SCALE_TYPE_DETECTION_TABLE = f"{CATALOG}.{SCHEMA}.scale_type_detection"

def validate_configuration():
    print("--- Validating configuration ---")
    error_found = False
    for var_name, var_value in [("CATALOG", CATALOG), ("SCHEMA", SCHEMA), ("TML_VOLUME", TML_VOLUME), ("LVDASH_VOLUME", LVDASH_VOLUME)]:
        if "/" in var_value or "\\" in var_value:
            print(f"ERROR: '{var_name}' contains a slash. Must be a single name.")
            error_found = True
    
    if not TML_INPUT_PATH.startswith("/Volumes/"):
        print(f"WARNING: 'TML_INPUT_PATH' does not look like a Volume path.")
        error_found = True
    
    if error_found:
        raise ValueError("Invalid configuration. Please review errors above.")
    else:
        print("Configuration looks good.")

validate_configuration()

def load_chart_type_mappings():
    """Load chart type mappings from configuration table - REQUIRED"""
    try:
        df = spark.table(CHART_TYPE_MAPPING_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {CHART_TYPE_MAPPING_TABLE} is empty. Please run the configuration setup notebook first.")
        
        mapping = {}
        for _, row in df.iterrows():
            tml_type = row['tml_chart_type']
            widget_type = row['widget_type']
            mapping[tml_type] = widget_type if pd.notna(widget_type) else None
        
        print(f"✓ Loaded {len(mapping)} chart type mappings from {CHART_TYPE_MAPPING_TABLE}")
        return mapping
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {CHART_TYPE_MAPPING_TABLE}

Details: {str(e)}

SOLUTION: You must run the configuration setup notebook to create the required tables:
1. Open the 'Configuration Tables Setup' notebook
2. Run all cells to create: chart_type_mappings, widget_size_config, expression_transformations, scale_type_detection
3. Then re-run this converter

Configuration tables are REQUIRED and no fallback mappings are available.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {CHART_TYPE_MAPPING_TABLE} not found or cannot be loaded. Run configuration setup first.") from e

def load_expression_transformations():
    """Load expression transformation rules from configuration table - REQUIRED"""
    try:
        df = spark.table(EXPRESSION_TRANSFORM_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {EXPRESSION_TRANSFORM_TABLE} is empty. Please run the configuration setup notebook first.")
        
        transformations = []
        for _, row in df.iterrows():
            transformations.append({
                'pattern': row['tml_pattern'],
                'target': row['target_expression']
            })
        print(f"✓ Loaded {len(transformations)} expression transformations from {EXPRESSION_TRANSFORM_TABLE}")
        return transformations
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {EXPRESSION_TRANSFORM_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {EXPRESSION_TRANSFORM_TABLE} not found. Run configuration setup first.") from e

def load_scale_type_rules():
    """Load scale type detection rules from configuration table - REQUIRED"""
    try:
        df = spark.table(SCALE_TYPE_DETECTION_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {SCALE_TYPE_DETECTION_TABLE} is empty. Please run the configuration setup notebook first.")
        
        rules = []
        for _, row in df.iterrows():
            rules.append({
                'pattern': row['field_pattern'],
                'scale_type': row['scale_type']
            })
        print(f"✓ Loaded {len(rules)} scale type rules from {SCALE_TYPE_DETECTION_TABLE}")
        return rules
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {SCALE_TYPE_DETECTION_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {SCALE_TYPE_DETECTION_TABLE} not found. Run configuration setup first.") from e

def load_widget_size_config():
    """Load widget size configuration from database table - REQUIRED"""
    try:
        df = spark.table(WIDGET_SIZE_CONFIG_TABLE).toPandas()
        
        if len(df) == 0:
            raise ValueError(f"Configuration table {WIDGET_SIZE_CONFIG_TABLE} is empty. Please run the configuration setup notebook first.")
        
        size_map = {}
        for _, row in df.iterrows():
            size_map[row['size_category']] = {
                'width': row['width'],
                'height': row['height']
            }
        print(f"✓ Loaded {len(size_map)} widget size configurations from {WIDGET_SIZE_CONFIG_TABLE}")
        return size_map
    except Exception as e:
        error_msg = f"""
ERROR: Failed to load required configuration table: {WIDGET_SIZE_CONFIG_TABLE}

Details: {str(e)}

SOLUTION: Run the configuration setup notebook to create required tables.
"""
        print(error_msg)
        raise RuntimeError(f"Required configuration table {WIDGET_SIZE_CONFIG_TABLE} not found. Run configuration setup first.") from e

# Load all configuration data
TML_TO_LVDASH_MAPPING = load_chart_type_mappings()
WIDGET_SIZE_MAP = load_widget_size_config()
EXPRESSION_TRANSFORMATIONS = load_expression_transformations()
SCALE_TYPE_RULES = load_scale_type_rules()


def load_mapping_data():
    try:
        mapping_df = spark.table(MAPPING_TABLE).toPandas()
        print(f"Loaded {len(mapping_df)} mappings from {MAPPING_TABLE}")
        
        unmapped = mapping_df[
            (mapping_df['databricks_table_name_ToBeFilled'].isna()) | 
            (mapping_df['databricks_table_name_ToBeFilled'] == '')
        ]
        
        if len(unmapped) > 0:
            print(f"\nWARNING: {len(unmapped)} visualizations are not mapped to Databricks tables:")
            for _, row in unmapped.head(10).iterrows():
                print(f"  - {row['tml_file']}: {row['visualization_name']}")
            if len(unmapped) > 10:
                print(f"  ... and {len(unmapped) - 10} more")
            print("\nThese visualizations will use TML table names directly.")
        
        return mapping_df
    except Exception as e:
        print(f"WARNING: Could not load mapping table: {e}")
        print("Conversion will proceed using TML table names directly.")
        return None

MAPPING_DATA = load_mapping_data()

def setup_environment():
    def ensure_schema_exists(catalog, schema):
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{schema}`")
    
    def ensure_volume_exists(catalog, schema, volume):
        try:
            spark.sql(f"DESCRIBE VOLUME `{catalog}`.`{schema}`.`{volume}`")
        except Exception:
            print(f"  Creating volume {volume}...")
            spark.sql(f"CREATE VOLUME IF NOT EXISTS `{catalog}`.`{schema}`.`{volume}`")
    
    def ensure_table_exists(full_table_name, table_type):
        parts = full_table_name.split('.')
        catalog = parts[0]
        schema = parts[1]
        table_name = parts[2]
        
        try:
            spark.sql(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{table_name}`")
            print(f"  Dropped existing table: {table_name}")
        except Exception:
            pass
        
        if table_type == "tracker":
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING, 
                    widget_name STRING, 
                    tml_type STRING, 
                    lvdash_type STRING, 
                    status STRING, 
                    available_fields STRING, 
                    missing_fields STRING, 
                    unmapped_properties STRING, 
                    notes STRING, 
                    conversion_timestamp TIMESTAMP
                ) USING DELTA
            """
        elif table_type == "failure":
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING,
                    error_type STRING,
                    error_message STRING,
                    stack_trace STRING,
                    failure_timestamp TIMESTAMP
                ) USING DELTA
            """
        else:
            create_sql = f"""
                CREATE OR REPLACE TABLE `{catalog}`.`{schema}`.`{table_name}` (
                    tml_file STRING, 
                    lvdash_file STRING, 
                    status STRING, 
                    num_datasets INT, 
                    num_pages INT, 
                    num_widgets INT, 
                    conversion_timestamp TIMESTAMP
                ) USING DELTA
            """
        
        spark.sql(create_sql)
        print(f"  Created table: {table_name}")
    
    print("\n--- Setting up environment ---")
    ensure_schema_exists(CATALOG, SCHEMA)
    ensure_volume_exists(CATALOG, SCHEMA, TML_VOLUME)
    ensure_volume_exists(CATALOG, SCHEMA, LVDASH_VOLUME)
    ensure_table_exists(TRACKER_TABLE, "tracker")
    ensure_table_exists(SUMMARY_TABLE, "summary")
    ensure_table_exists(FAILURE_TABLE, "failure")
    print("--- Environment ready ---\n")

def generate_unique_id(length=8):
    return uuid.uuid4().hex[:length]

def parse_tml_file(file_path):
    try:
        content = dbutils.fs.head(file_path, 10 * 1024 * 1024)
        try:
            return yaml.safe_load(content)
        except yaml.YAMLError:
            return json.loads(content)
    except Exception as e:
        raise ValueError(f"Failed to parse TML file: {str(e)}")

def extract_colors_from_tml(chart_data):
    try:
        client_state = chart_data.get('client_state_v2', '{}')
        if isinstance(client_state, str):
            client_state = json.loads(client_state)
        
        system_colors = client_state.get('systemSeriesColors', [])
        if system_colors:
            colors = []
            for item in system_colors:
                if item.get('color') and item['color'] not in colors:
                    colors.append(item['color'])
            if colors:
                return colors
    except Exception:
        pass
    
    return ["#2E75F0", "#06BF7F", "#FCC838", "#48D1E0", "#71A1F4", "#8C62F5"]

def infer_scale_type(column_name):
    """Infer scale type using configuration rules"""
    name = column_name.lower()
    
    # Use configuration rules if available
    if SCALE_TYPE_RULES:
        for rule in SCALE_TYPE_RULES:
            pattern = rule['pattern'].lower()
            # Remove wildcards and check if pattern is in the column name
            pattern_parts = pattern.replace('*_', '').replace('_*', '').split(',')
            for part in pattern_parts:
                part = part.strip()
                if part in name:
                    return rule['scale_type']
    
    # Fallback to default logic
    if any(t in name for t in ['date', 'time', 'year', 'month', 'day', 'timestamp', 'week']):
        return 'temporal'
    
    if any(t in name for t in ['total', 'sum', 'count', 'avg', 'average', 'min', 'max', 'revenue', 'price', 'quantity', 'amount', 'sales', 'number']):
        return 'quantitative'
    
    return 'categorical'

def clean_field_name(field_name):
    if not field_name:
        return ""
    cleaned = re.sub(
        r'^(Total |sum\(|count\(|avg\(|min\(|max\(|Unique Number of )',
        '',
        field_name,
        flags=re.IGNORECASE
    )
    cleaned = re.sub(r'\)$', '', cleaned)
    return cleaned.strip()

def sanitize_alias_name(alias_name):
  if not alias_name:
    return ""
  # Replace sequences of one or more non-alphanumeric characters with a single underscore
  safe_name = re.sub(r'\W+', '_', alias_name)
  # Remove leading/trailing underscores that might result from the substitution
  sanitized_alias = safe_name.strip('_')
  # Fallback if the name becomes empty after stripping (e.g., input was just '---')
  return sanitized_alias if sanitized_alias else "_"

class ConversionTracker:
    def __init__(self):
        self.records = []
    
    def add_record(self, tml_file, widget_name, tml_type, lvdash_type, status, available, missing, unmapped_props, notes=""):
        self.records.append({
            'tml_file': tml_file, 
            'widget_name': widget_name, 
            'tml_type': tml_type,
            'lvdash_type': lvdash_type, 
            'status': status,
            'available_fields': ', '.join(available), 
            'missing_fields': ', '.join(missing),
            'unmapped_properties': ', '.join(unmapped_props),
            'notes': notes, 
            'conversion_timestamp': datetime.now()
        })

class TMLToLVDASHConverter:
    def __init__(self, tml_data, tml_filename, tracker, mapping_data=None):
        self.tml_data = tml_data
        self.tml_filename = tml_filename
        self.tracker = tracker
        self.mapping_data = mapping_data
        
    def _get_mapping_for_viz(self, viz_id):
        """Get mapping for a specific visualization ID, returns Series or None"""
        if self.mapping_data is None or len(self.mapping_data) == 0:
            return None
        
        try:
            mapping = self.mapping_data[
                (self.mapping_data['tml_file'] == self.tml_filename) &
                (self.mapping_data['visualization_id'] == viz_id)
            ]
            
            # Always return a Series (single row) or None
            if not mapping.empty:
                return mapping.iloc[0]  # Returns Series
            return None
        except Exception as e:
            print(f"  WARNING: Error getting mapping for viz {viz_id}: {e}")
            return None

    def convert(self):
        liveboard = self.tml_data.get('liveboard')
        if not liveboard:
            raise ValueError("TML file missing 'liveboard' root key.")

        visualizations = liveboard.get('visualizations', [])
        if not visualizations:
            raise ValueError("No visualizations found in liveboard.")
        
        # Track unique datasets by their name (supports reuse)
        datasets_map = {}  # Key: dataset_name, Value: dataset object
        dataset_usage = {}  # Key: dataset_name, Value: list of viz names using it
        widgets = []

        for viz in visualizations:
            try:
                viz_id = viz.get('id')
                viz_name = viz.get('answer', {}).get('name', 'Unknown')
                
                # Check if this viz should use a common/shared dataset
                mapping = self._get_mapping_for_viz(viz_id)
                print("Check1", mapping)
                
                if mapping is not None:
                    common_ds = mapping.get('common_dataset_name')
                    # Check if common_dataset_name is truly populated
                    if pd.notna(common_ds) and common_ds and str(common_ds).strip() != '' and str(common_ds).lower() != 'null':
                        # USE SHARED DATASET from mapping table
                        dataset_name = common_ds
                        # Create the shared dataset only once
                        if dataset_name not in datasets_map:
                            dataset = self._create_shared_dataset(viz, mapping)
                            datasets_map[dataset_name] = dataset
                            dataset_usage[dataset_name] = []
                            print(f"  INFO: Created shared dataset '{dataset_name}' for visualization '{viz_name}'")
                        else:
                            print(f"  INFO: Reusing shared dataset '{dataset_name}' for visualization '{viz_name}'")
                        
                        dataset_usage[dataset_name].append(viz_name)
                    else:
                        # CREATE UNIQUE DATASET per visualization
                        dataset = self._create_dataset(viz)
                        dataset_name = dataset['name']
                        datasets_map[dataset_name] = dataset
                        dataset_usage[dataset_name] = [viz_name]
                        print(f"  INFO: Created unique dataset '{dataset_name}' for visualization '{viz_name}'")
                else:
                    # CREATE UNIQUE DATASET per visualization (no mapping found)
                    dataset = self._create_dataset(viz)
                    dataset_name = dataset['name']
                    datasets_map[dataset_name] = dataset
                    dataset_usage[dataset_name] = [viz_name]
                    print(f"  INFO: Created unique dataset '{dataset_name}' for visualization '{viz_name}'")
                        
                # Create widget using the appropriate dataset name
                widget_data = self._create_widget(viz, dataset_name)
                
                # CRITICAL: Handle both single widget and multiple widgets
                # _create_widget returns either:
                #   - A dict (single widget)
                #   - A list of dicts (multiple widgets, e.g., counter + trend)
                if isinstance(widget_data, list):
                    # Multiple widgets - extend the list
                    for w in widget_data:
                        # Defensive check - ensure each item is a dict
                        if isinstance(w, dict):
                            widgets.append(w)
                        else:
                            print(f"  WARNING: Skipping invalid widget object of type {type(w)}")
                elif isinstance(widget_data, dict):
                    # Single widget - append directly
                    widgets.append(widget_data)
                else:
                    print(f"  WARNING: _create_widget returned unexpected type {type(widget_data)} for viz {viz_id}")
                
            except Exception as viz_error:
                viz_name = viz.get('answer', {}).get('name', 'Unknown')
                print(f"  WARNING: Failed to convert visualization '{viz_name}': {viz_error}")
                import traceback
                traceback.print_exc()
                self.tracker.add_record(
                    self.tml_filename,
                    viz_name,
                    viz.get('answer', {}).get('chart', {}).get('type', 'UNKNOWN'),
                    'ERROR',
                    'ERROR',
                    [],
                    [],
                    [],
                    f"Conversion failed: {str(viz_error)[:500]}"
                )
                continue

        if not widgets:
            raise ValueError("No widgets could be converted successfully.")

        # Log dataset reuse statistics
        print(f"\n--- Dataset Reuse Summary ---")
        for ds_name, viz_list in dataset_usage.items():
            if len(viz_list) > 1:
                print(f"  ✓ Shared dataset '{ds_name}' used by {len(viz_list)} visualizations: {', '.join(viz_list[:3])}{'...' if len(viz_list) > 3 else ''}")
        print(f"Total datasets created: {len(datasets_map)}")
        print(f"Total widgets created: {len(widgets)}")
        
        # DEBUG: Check widget structure
        print(f"\n--- Widget Structure Check ---")
        for i, w in enumerate(widgets):
            if not isinstance(w, dict):
                print(f"  ERROR: Widget at index {i} is type {type(w)}, not dict!")
            elif 'widget' not in w:
                print(f"  WARNING: Widget at index {i} missing 'widget' key")
            else:
                print(f"  ✓ Widget {i}: viz_id={w.get('viz_id')}, type={w.get('widget', {}).get('spec', {}).get('widgetType')}")
        print("="*50 + "\n")

        return {
            "datasets": list(datasets_map.values()),
            "pages": [{
                "name": generate_unique_id(),
                "displayName": liveboard.get('name', 'Converted Dashboard'),
                "layout": self._create_layout(liveboard.get('layout'), widgets),
                "pageType": "PAGE_TYPE_CANVAS"
            }]
        }

    def _translate_search_query_to_sql(self, viz):
        answer = viz.get('answer', {})
        query = answer.get('search_query', '')
        viz_id = viz.get('id')
        print("Viz_id", viz_id)
        print("Query", query)
        print("Answer", answer)

        mapping = self._get_mapping_for_viz(viz_id)
        
        if mapping is not None and mapping.get('databricks_table_name_ToBeFilled'):
            table_name = mapping['databricks_table_name_ToBeFilled']
        else:
            table_name = answer.get('tables', [{}])[0].get('name', 'your_source_table')
        
        column_mapping = {}
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                column_mapping = {}
        
        answer_cols = [col.get('name') for col in answer.get('answer_columns', []) if col.get('name')]
        
        select_clauses = []
        group_by_cols = []
        
        for i, col_name in enumerate(answer_cols, 1):
            if not col_name:
                continue
            
            base_field = clean_field_name(col_name)
            mapped_field = column_mapping.get(base_field, base_field)
            mapped_col_name = column_mapping.get(col_name, col_name)
            sanitized_alias = sanitize_alias_name(mapped_col_name)
            sanitized_base_alias = sanitize_alias_name(base_field) 

            print("Fields : base mapped mapped_col sanitized_alias", base_field, " ", mapped_field, " ", mapped_col_name, " ", sanitized_alias)


            if re.match(r"(Day|Week|Month|Year)\(", col_name):
                match = re.search(r"(\w+)\((.*?)\)", col_name)
                if match:
                    func, field = match.groups()
                    db_field = column_mapping.get(field, field)
                    select_clauses.append(f"  DATE_TRUNC('{func.upper()}', {db_field}) AS {sanitized_alias}")
                    group_by_cols.append(str(i))
            
            elif 'Unique Number of' in col_name or 'unique number of' in col_name.lower():
                select_clauses.append(f"  COUNT(DISTINCT {mapped_field}) AS {sanitized_alias}")
            
            elif any(p in col_name for p in ['Total ', 'sum(', 'count(', 'Sum(', 'Count(']):
                agg = 'COUNT' if 'count' in col_name.lower() else 'SUM'
                select_clauses.append(f"  {agg}({mapped_field}) AS {sanitized_base_alias}")
            
            else:
                select_clauses.append(f"  {mapped_field} AS {sanitized_base_alias}")
                group_by_cols.append(str(i))

        if not select_clauses:
            return [f"SELECT '' AS placeholder FROM {table_name}"]

        where_clauses = []
        if ". 'this month'" in query:
            where_clauses.append("DATE >= DATE_TRUNC('MONTH', CURRENT_TIMESTAMP())")
        if ". 'last 30 days'" in query:
            where_clauses.append("DATE >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)")

        order_by = ""
        limit = ""
        if 'top 10' in query.lower():
            measure_idx = next((i for i, col in enumerate(answer_cols, 1) if any(p in col for p in ['Total', 'Unique', 'sum(', 'count('])), 1)
            order_by = f" ORDER BY {measure_idx} DESC"
            limit = " LIMIT 10"

        sql_parts = ["SELECT", ",\n".join(select_clauses), f" FROM {table_name} "]
        
        if where_clauses:
            sql_parts.append(f" WHERE {' AND '.join(where_clauses)} ")
        
        has_agg = any(agg in ' '.join(select_clauses) for agg in ['SUM(', 'COUNT(', 'AVG('])
        if group_by_cols and has_agg:
            sql_parts.append(f" GROUP BY {', '.join(group_by_cols)} ")
        
        if order_by:
            sql_parts.append(order_by)
        if limit:
            sql_parts.append(limit)
        
        return '\n'.join(sql_parts).split('\n')

    def _create_dataset(self, viz):
        answer = viz.get('answer', {})
        print("create dataset inside", f"ds_{viz.get('id', generate_unique_id())}" , " ", answer.get('name', 'Untitled Dataset')," ", self._translate_search_query_to_sql(viz)  )


        return {
            "name": f"ds_{viz.get('id', generate_unique_id())}",
            "displayName": answer.get('name', 'Untitled Dataset'),
            "queryLines": self._translate_search_query_to_sql(viz)
        }

    def _extract_fields_from_answer(self, answer, viz_id,widget_type=None):
        mapping = self._get_mapping_for_viz(viz_id)
        
        # Determine which column mapping to use
        column_mapping = {}
        if mapping is not None:
            # PRIORITY 1: Use common_column_mapping if this is a shared dataset
            common_dataset = mapping.get('common_dataset_name')
            common_mapping = mapping.get('common_column_mapping')
            if pd.notna(common_dataset) and pd.notna(common_mapping):
                try:
                    column_mapping = json.loads(mapping['common_column_mapping'])
                    print(f"  INFO: Using common_column_mapping for viz {viz_id}")
                except (json.JSONDecodeError, TypeError):
                    print(f"  WARNING: Failed to parse common_column_mapping for viz {viz_id}")
            
            # PRIORITY 2: Fall back to regular column mapping
            if not column_mapping and mapping.get('databricks_column_mapping_ToBeFilled'):
                try:
                    column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
                    print(f"  INFO: Using databricks_column_mapping for viz {viz_id}")
                except (json.JSONDecodeError, TypeError):
                    pass
        
        fields = []
        for col in answer.get('answer_columns', []):
            col_name = col.get('name')
            if not col_name:
                continue
            
            # Use the same logic as SQL generation to determine the alias
            base_field = clean_field_name(col_name)
            sql_alias = sanitize_alias_name(base_field)
            
            mapped_field = column_mapping.get(base_field, base_field)
            
            # USE CONFIGURATION-DRIVEN EXPRESSION TRANSFORMATION
            expression = self._apply_expression_transformation(col_name, sql_alias, base_field, column_mapping)
            
            fields.append({
                "name": sql_alias,
                "expression": expression
            })
            
            print(f"  Field created: name={sql_alias}, expression={expression}")
        
        return fields


    def _apply_expression_transformation(self, col_name, sql_alias, base_field, column_mapping):
        """
        Apply expression transformation using configuration from EXPRESSION_TRANSFORMATIONS.
        
        Args:
            col_name: Original column name from TML (e.g., "Day(Event Date)", "Total Revenue")
            sql_alias: Sanitized SQL alias (e.g., "Event_Date", "Revenue")
            base_field: Cleaned field name
            column_mapping: Dictionary of column mappings
            
        Returns:
            str: Transformed expression (e.g., "DATE_TRUNC('DAY', Event_Date)", "SUM(Revenue)")
        """
        import re
        
        # Try to match against configured expression patterns
        for transform in EXPRESSION_TRANSFORMATIONS:
            pattern = transform['pattern']
            target_expr = transform['target']
            
            # Handle date functions: Day(field), Month(field), etc.
            if pattern.startswith(('Day(', 'Week(', 'Month(', 'Year(')):
                func_match = re.match(r"(Day|Week|Month|Year)\((.*?)\)", col_name, re.IGNORECASE)
                if func_match:
                    func_name, field_name = func_match.groups()
                    # Get the mapped field from column_mapping
                    db_field = column_mapping.get(field_name, field_name)
                    sanitized_field = sanitize_alias_name(base_field)
                    
                    # Replace 'field' placeholder with actual field name
                    # target_expr example: "DATE_TRUNC('DAY', `field`)"
                    expression = target_expr.replace('field', sanitized_field)
                    expression = expression.replace('DAY', func_name.upper())
                    expression = expression.replace('WEEK', func_name.upper())
                    expression = expression.replace('MONTH', func_name.upper())
                    expression = expression.replace('YEAR', func_name.upper())
                    
                    return expression
            
            # Handle aggregation functions: sum(field), count(field), etc.
            # SKIP THIS - Don't add SUM() wrapper for any fields
            # The SQL query already has the aggregation
            elif pattern.startswith(('sum(', 'count(', 'avg(', 'min(', 'max(')):
                # Just skip aggregation patterns - return field as-is
                continue
        
        # No transformation matched - return as-is (direct field reference)
        return f'`{sql_alias}`'
        
    def _create_shared_dataset(self, viz, mapping):
        """
        Create a shared dataset using common SQL and column mapping from the mapping table.
        
        Args:
            viz: Visualization object (used for fallback display name)
            mapping: Mapping row with common_dataset_name, common_sql_query, common_column_mapping
            
        Returns:
            dict: Dataset object with name, displayName, and queryLines
        """
        dataset_name = mapping['common_dataset_name']
        
        # Get SQL query from mapping table
        if mapping.get('common_sql_query'):
            sql_query = mapping['common_sql_query']
            # Split into lines for queryLines format
            query_lines = sql_query.strip().split('\n')
        else:
            # Fallback: generate SQL from viz if common_sql_query is empty
            print(f"  WARNING: No common_sql_query found for shared dataset '{dataset_name}', generating from viz")
            query_lines = self._translate_search_query_to_sql(viz)
        
        # Get display name from mapping or viz
        display_name = mapping.get('common_dataset_name', viz.get('answer', {}).get('name', 'Shared Dataset'))
        
        print(f"  INFO: Shared dataset '{dataset_name}' created with {len(query_lines)} query lines")
        
        return {
            "name": dataset_name,
            "displayName": display_name,
            "queryLines": query_lines
        }
        
    def _create_widget(self, viz, dataset_name):
        answer = viz.get('answer', {})
        chart = answer.get('chart', {})
        
        display_mode = answer.get('display_mode', '')
        tml_type = chart.get('type', 'TABLE_MODE' if display_mode == 'TABLE_MODE' else 'UNKNOWN')
        
        lvdash_type = TML_TO_LVDASH_MAPPING.get(tml_type)
        
        if lvdash_type is None:
            lvdash_type = 'table'
            status_note = f"Chart type '{tml_type}' has no direct mapping, using fallback"
        else:
            status_note = ""
        
        available = []
        missing = []
        unmapped_props = []
        
        spec_builder = getattr(self, f'_get_{lvdash_type}_spec', self._get_table_spec)
        
        # Get the spec (might be a tuple for KPI with trend)
        spec_result = spec_builder(answer, chart, available, missing, unmapped_props, viz.get('id'))
        
        # Check if result is a tuple (counter + line chart)
        if isinstance(spec_result, tuple):
            counter_spec, line_spec = spec_result
            
            # Log the counter widget
            self.tracker.add_record(
                self.tml_filename, 
                answer.get('name', 'Unnamed'), 
                tml_type, 
                'counter',
                'COMPLETE' if not missing and not unmapped_props else 'PARTIAL', 
                available, 
                missing,
                unmapped_props,
                status_note + " (with trend line)"
            )
            
            fields = self._extract_fields_from_answer(answer, viz.get('id'))
            
            # Create counter widget
            counter_widget = {
                "viz_id": viz.get('id'),
                "viz_guid": viz.get('viz_guid'),
                "widget": {
                    "name": f"widget_{viz.get('id', generate_unique_id())}",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False
                        }
                    }],
                    "spec": counter_spec
                }
            }
            
            # Create line chart widget for trend
            line_widget = {
                "viz_id": f"{viz.get('id')}_trend",
                "viz_guid": viz.get('viz_guid'),
                "is_trend_chart": True,  # Flag to identify this as a trend chart
                "parent_viz_id": viz.get('id'),  # Link to parent counter
                "widget": {
                    "name": f"widget_{viz.get('id', generate_unique_id())}_trend",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False
                        }
                    }],
                    "spec": line_spec
                }
            }
            
            # Return LIST of two widgets
            return [counter_widget, line_widget]
        
        # Normal single widget handling
        self.tracker.add_record(
            self.tml_filename, 
            answer.get('name', 'Unnamed'), 
            tml_type, 
            lvdash_type,
            'COMPLETE' if not missing and not unmapped_props else 'PARTIAL', 
            available, 
            missing,
            unmapped_props,
            status_note
        )
        
        fields = self._extract_fields_from_answer(answer, viz.get('id'))
        
        return {
            "viz_id": viz.get('id'),
            "viz_guid": viz.get('viz_guid'),
            "widget": {
                "name": f"widget_{viz.get('id', generate_unique_id())}",
                "queries": [{
                    "name": "main_query",
                    "query": {
                        "datasetName": dataset_name,
                        "fields": fields,
                        "disaggregated": False
                    }
                }],
                "spec": spec_result  # Use spec_result instead of spec
            }
        }

    def _get_base_frame(self, answer):
        return {
            "title": answer.get('name', 'Untitled'),
            "description": answer.get('description', ''),
            "showTitle": True,
            "showDescription": bool(answer.get('description'))
        }

    def _get_bar_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """Generates bar/column chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        colors = extract_colors_from_tml(chart) # Assumes helper exists
        mark = {}

        tml_type = chart.get('type', '')
        is_column = 'COLUMN' in tml_type.upper()
        is_stacked = 'STACKED' in tml_type.upper()

        print(f"\n=== {'COLUMN' if is_column else 'BAR'} CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}, Is Column: {is_column}, Is Stacked: {is_stacked}")

        # --- NEW: Extract Custom Axis Names ---
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")
        # --- END NEW BLOCK ---

        # --- Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_mapping = {} # Holds the JSON mapping dict if found
        if mapping is not None:
            # Logic to load either common or specific mapping JSON
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: 
                    column_mapping = json.loads(mapping_str_to_use)
                    print(f"  Column mapping loaded: {column_mapping}")
                except (json.JSONDecodeError, TypeError) as e: 
                    print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: 
            print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        # --- Determine Axis Roles ---
        if is_column: category_axis_key, measure_axis_key = 'x', 'y'
        else: category_axis_key, measure_axis_key = 'y', 'x'

        tml_category_config = axis_config.get('x', [])
        tml_measure_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"Category Axis Key: {category_axis_key}, Measure Axis Key: {measure_axis_key}")
        print(f"TML Category Config: {tml_category_config}")
        print(f"TML Measure Config: {tml_measure_config}")
        print(f"TML Color Config: {tml_color_config}")

        # --- Process Category Axis (Dimension) ---
        if tml_category_config:
            original_field_cat = tml_category_config[0] # << CAPTURE ORIGINAL NAME FROM TML CONFIG
            matching_cat_col = next((col for col in answer_cols if col.get('name') == original_field_cat or clean_field_name(col.get('name')) == clean_field_name(original_field_cat)), None)

            if matching_cat_col:
                cat_col_name_from_answer = matching_cat_col.get('name') # Use this for type inference
                cat_base_field = clean_field_name(cat_col_name_from_answer)
                # !! CORRECTED: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                cat_sql_alias = column_mapping.get(original_field_cat, sanitize_alias_name(cat_base_field))
                print(f"Category axis ({category_axis_key}): TML={original_field_cat}, FoundInAnswer={cat_col_name_from_answer}, MappedSQLAlias={cat_sql_alias}")

                encodings[category_axis_key] = {
                    "fieldName": cat_sql_alias,                # Use mapped/sanitized name for data binding
                    "displayName": original_field_cat,         # Use original TML name for display
                    "scale": {"type": infer_scale_type(cat_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_cat} # Use custom or original name
                }
                available.append(f'{category_axis_key}-axis (category)')
            else:
                print(f"Category axis WARNING: No match in answer_cols for TML field '{original_field_cat}'")
                missing.append(f'{category_axis_key}-axis (category)')
        else:
             print(f"Category axis WARNING: No TML config found.")
             missing.append(f'{category_axis_key}-axis (category)')

        # --- Process Measure Axis (One or More Measures) ---
        measure_fields_processed = [] # Holds {"fieldName": "sql_alias", "originalName": "original_field_measure"}
        original_measure_names = []   # Store original names from TML config for axis title
        measure_axis_title = "Value"  # Default title

        if tml_measure_config:
            print(f"Processing {len(tml_measure_config)} measure field(s)...")
            for original_field_measure in tml_measure_config: # << CAPTURE ORIGINAL NAME FROM TML CONFIG
                original_measure_names.append(original_field_measure)
                matching_measure_col = next((col for col in answer_cols if col.get('name') == original_field_measure or clean_field_name(col.get('name')) == clean_field_name(original_field_measure)), None)

                if matching_measure_col:
                    measure_col_name_from_answer = matching_measure_col.get('name')
                    measure_base_field = clean_field_name(measure_col_name_from_answer)
                    # !! CORRECTED: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                    measure_sql_alias = column_mapping.get(original_field_measure, sanitize_alias_name(measure_base_field))
                    print(f"  - Measure field: TML={original_field_measure}, FoundInAnswer={measure_col_name_from_answer}, MappedSQLAlias={measure_sql_alias}")
                    measure_fields_processed.append({"fieldName": measure_sql_alias, "originalName": original_field_measure})
                else:
                    print(f"Measure axis WARNING: No match in answer_cols for TML field '{original_field_measure}'")
                    missing.append(f'{measure_axis_key}-axis measure: {original_field_measure}')

            # --- Construct Measure Axis Encoding ---
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                measure_axis_title = custom_y_axis_title or measure_info["originalName"] # << USE CUSTOM/ORIGINAL NAME
                encodings[measure_axis_key] = {
                    "fieldName": measure_info["fieldName"],    # Use mapped/sanitized alias
                    "displayName": measure_info["originalName"], # Use original TML name
                    "scale": {"type": "quantitative"},
                    "axis": {"title": measure_axis_title}      # Use custom or original name
                }
                available.append(f'{measure_axis_key}-axis (single measure)')
                print(f"Measure axis ({measure_axis_key}): Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                measure_axis_title = custom_y_axis_title or ", ".join(original_measure_names[:3]) + ("..." if len(original_measure_names) > 3 else "") # << USE CUSTOM/ORIGINAL NAME
                encodings[measure_axis_key] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], # Use mapped/sanitized aliases
                    "scale": {"type": "quantitative"},
                    "axis": {"title": measure_axis_title} # Use custom or combined original TML names
                }
                available.append(f'{measure_axis_key}-axis ({len(measure_fields_processed)} measures)')
                print(f"Measure axis ({measure_axis_key}): Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: 
                 if not any(m.startswith(f'{measure_axis_key}-axis measure:') for m in missing):
                    missing.append(f'{measure_axis_key}-axis (measure)')
        else:
             print(f"Measure axis WARNING: No TML config found.")
             missing.append(f'{measure_axis_key}-axis (measure)')

        # --- Process Color (Grouping/Stacking Dimension) ---
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0] # << CAPTURE ORIGINAL NAME FROM TML CONFIG
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                
                # !! CORRECTED: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,        # Use mapped/sanitized alias
                    "displayName": original_field_color, # Use original TML name
                    "scale": {"type": "categorical"}
                }
                
                color_dimension_present = True
                available.append('color')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             print("Color axis: No color dimension specified in TML.")

        # --- Determine Stacking/Grouping in `mark` ---
        if is_stacked:
            if color_dimension_present: 
                mark = {"stackType": "stacked"}
                available.append('stacking (by dimension)')
                print("Mark: Stacking by color dimension")
            elif len(measure_fields_processed) > 1: 
                mark = {"stackType": "stacked"}
                available.append('stacking (multiple measures)')
                print("Mark: Stacking by multiple measures")
            else: 
                print("Mark: Single measure, no color dimension - No stacking needed.")
        else: # Not stacked
            if color_dimension_present: 
                mark = {"stackType": "grouped"}
                available.append('grouping (by dimension)')
                print("Mark: Grouping by color dimension")

        if not mark and not color_dimension_present and colors:
             print(f"Mark: Applying first TML color (fallback): {colors[0]}")
             mark = {"colors": colors[:1]}

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        return { "widgetType": "bar", "version": 3, "frame": self._get_base_frame(answer), "encodings": encodings, "mark": mark }

    def _get_line_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """Generates line chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        mark = {} 
        tml_type = chart.get('type', '')
        is_stacked_area = 'STACKED_AREA' in tml_type.upper()

        print(f"\n=== LINE/AREA CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}")

        # --- NEW: Extract Custom Axis Names ---
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")
        # --- END NEW BLOCK ---

        # --- Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_mapping = {}
        if mapping is not None:
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: column_mapping = json.loads(mapping_str_to_use)
                except (json.JSONDecodeError, TypeError) as e: print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        # --- TML Config Extraction ---
        tml_x_config = axis_config.get('x', [])
        tml_y_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"TML X Config: {tml_x_config}")
        print(f"TML Y Config: {tml_y_config}")
        print(f"TML Color Config: {tml_color_config}")

        # --- Process X-axis (Dimension - usually temporal) ---
        if tml_x_config:
            original_field_x = tml_x_config[0] # << CAPTURE ORIGINAL NAME
            matching_x_col = next((col for col in answer_cols if col.get('name') == original_field_x or clean_field_name(col.get('name')) == clean_field_name(original_field_x)), None)

            if matching_x_col:
                x_col_name_from_answer = matching_x_col.get('name')
                x_base_field = clean_field_name(x_col_name_from_answer)
                # !! CORRECTED LOGIC: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                x_sql_alias = column_mapping.get(original_field_x, sanitize_alias_name(x_base_field))
                print(f"X-axis: TML={original_field_x}, FoundInAnswer={x_col_name_from_answer}, MappedSQLAlias={x_sql_alias}")

                encodings['x'] = {
                    "fieldName": x_sql_alias,
                    "displayName": original_field_x, # Use original TML name
                    "scale": {"type": infer_scale_type(x_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_x} # << USE CUSTOM/ORIGINAL NAME
                }
                available.append('x-axis')
            else:
                print(f"X-axis WARNING: No match in answer_cols for TML field '{original_field_x}'")
                missing.append('x-axis')
        else:
            print(f"X-axis WARNING: No TML config found.")
            missing.append('x-axis')

        # --- Process Y-axis (One or More Measures) ---
        measure_fields_processed = [] 
        original_y_names = []         
        y_axis_title = "Value"        

        if tml_y_config:
            print(f"Processing {len(tml_y_config)} Y-axis measure field(s)...")
            for original_field_y in tml_y_config: # << CAPTURE ORIGINAL NAME
                original_y_names.append(original_field_y)
                matching_y_col = next((col for col in answer_cols if col.get('name') == original_field_y or clean_field_name(col.get('name')) == clean_field_name(original_field_y)), None)

                if matching_y_col:
                    y_col_name_from_answer = matching_y_col.get('name')
                    y_base_field = clean_field_name(y_col_name_from_answer)
                    # !! CORRECTED LOGIC: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                    y_sql_alias = column_mapping.get(original_field_y, sanitize_alias_name(y_base_field))
                    print(f"  - Y-axis measure: TML={original_field_y}, FoundInAnswer={y_col_name_from_answer}, MappedSQLAlias={y_sql_alias}")
                    measure_fields_processed.append({"fieldName": y_sql_alias, "originalName": original_field_y})
                else:
                    print(f"Y-axis WARNING: No match in answer_cols for TML field '{original_field_y}'")
                    missing.append(f'y-axis measure: {original_field_y}')

            # --- Construct Y-axis Encoding ---
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                y_axis_title = custom_y_axis_title or measure_info["originalName"] # << USE CUSTOM/ORIGINAL NAME
                encodings['y'] = {
                    "fieldName": measure_info["fieldName"],    
                    "displayName": measure_info["originalName"], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title}          
                }
                available.append('y-axis (single measure)')
                print(f"Y-axis: Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                y_axis_title = custom_y_axis_title or ", ".join(original_y_names[:3]) + ("..." if len(original_y_names) > 3 else "") # << USE CUSTOM/ORIGINAL NAME
                encodings['y'] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title} # Use combined original TML names
                }
                available.append(f'y-axis ({len(measure_fields_processed)} measures)')
                print(f"Y-axis: Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: # No valid measures found
                 if not any(m.startswith('y-axis measure:') for m in missing):
                    missing.append('y-axis (measure)')
        else:
             print(f"Y-axis WARNING: No TML config found.")
             missing.append('y-axis (measure)')

        # --- Process Color (Dimension for Multiple Lines) ---
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0] # << CAPTURE ORIGINAL NAME
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                # !! CORRECTED LOGIC: Look up ORIGINAL TML name in mapping, fallback to sanitizing the BASE name
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,
                    "displayName": original_field_color, # Use original TML name
                    "scale": {"type": "categorical"}
                }
                color_dimension_present = True
                available.append('color (series)')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             if len(measure_fields_processed) > 1:
                 # This is for Viz 4. Databricks will use the measure names for the legend.
                 print("Color axis: No TML config, Databricks will use multiple Y-fields for color.")
                 available.append('color (implicit from measures)')
             else:
                 print("Color axis: No color dimension specified in TML.")
        
        # Handle Stacked Area
        if is_stacked_area:
             if color_dimension_present or len(measure_fields_processed) > 1:
                 mark = {"stackType": "stacked"}
                 available.append('stacking (area)')
                 print("Mark: Stacking enabled for area chart")

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        widget_type = "area" if "AREA" in tml_type.upper() else "line"

        return {
            "widgetType": widget_type,
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings,
            "mark": mark 
        }

    def _create_widget(self, viz, dataset_name):
        answer = viz.get('answer', {})
        chart = answer.get('chart', {})
        
        display_mode = answer.get('display_mode', '')
        tml_type = chart.get('type', 'TABLE_MODE' if display_mode == 'TABLE_MODE' else 'UNKNOWN')
        
        lvdash_type = TML_TO_LVDASH_MAPPING.get(tml_type)
        
        if lvdash_type is None:
            lvdash_type = 'table'
            status_note = f"Chart type '{tml_type}' has no direct mapping, using fallback"
        else:
            status_note = ""
        
        available = []
        missing = []
        unmapped_props = []
        
        # Get spec builder function
        spec_builder = getattr(self, f'_get_{lvdash_type}_spec', self._get_table_spec)
        
        # Call spec builder and check if it returns a tuple (KPI with trend)
        spec_result = spec_builder(answer, chart, available, missing, unmapped_props, viz.get('id'))
        
        # Extract fields once (used by all widgets)
        fields = self._extract_fields_from_answer(answer, viz.get('id'))
        
        # Check if result is a tuple (counter + line chart for KPI with temporal data)
        if isinstance(spec_result, tuple) and len(spec_result) == 2:
            counter_spec, line_spec = spec_result
            
            # Log the counter widget
            self.tracker.add_record(
                self.tml_filename, 
                answer.get('name', 'Unnamed'), 
                tml_type, 
                'counter',
                'COMPLETE' if not missing and not unmapped_props else 'PARTIAL', 
                available, 
                missing,
                unmapped_props,
                status_note + " (with trend line)"
            )
            
            # Create counter widget
            counter_widget = {
                "viz_id": viz.get('id'),
                "viz_guid": viz.get('viz_guid'),
                "widget": {
                    "name": f"widget_{viz.get('id', generate_unique_id())}",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False
                        }
                    }],
                    "spec": counter_spec
                }
            }
            
            # Create line chart widget for trend
            line_widget = {
                "viz_id": f"{viz.get('id')}_trend",
                "viz_guid": viz.get('viz_guid'),
                "is_trend_chart": True,  # Flag to identify this as a trend chart
                "parent_viz_id": viz.get('id'),  # Link to parent counter
                "widget": {
                    "name": f"widget_{viz.get('id', generate_unique_id())}_trend",
                    "queries": [{
                        "name": "main_query",
                        "query": {
                            "datasetName": dataset_name,
                            "fields": fields,
                            "disaggregated": False
                        }
                    }],
                    "spec": line_spec
                }
            }
            
            # Return LIST of two widgets
            return [counter_widget, line_widget]
        
        # Normal single widget handling (spec_result is a dict)
        self.tracker.add_record(
            self.tml_filename, 
            answer.get('name', 'Unnamed'), 
            tml_type, 
            lvdash_type,
            'COMPLETE' if not missing and not unmapped_props else 'PARTIAL', 
            available, 
            missing,
            unmapped_props,
            status_note
        )
        
        # Return single widget (NOT in a list)
        return {
            "viz_id": viz.get('id'),
            "viz_guid": viz.get('viz_guid'),
            "widget": {
                "name": f"widget_{viz.get('id', generate_unique_id())}",
                "queries": [{
                    "name": "main_query",
                    "query": {
                        "datasetName": dataset_name,
                        "fields": fields,
                        "disaggregated": False
                    }
                }],
                "spec": spec_result  # ✅ This is a dict, not a tuple
            }
        }

    def _get_pie_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate pie chart specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Pie chart specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0] 
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process Y-axis (angle/value) - typically the measure
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['angle'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('angle')
            else:
                missing.append('angle')
        else:
            missing.append('angle')
        
        # Process X-axis (color/category) - typically the dimension
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color')
            else:
                missing.append('color')
        else:
            missing.append('color')
        
        encodings['label'] = {"show": True}
        
        return {
            "widgetType": "pie", 
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_area_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None): 
        """Generates area/stacked area chart spec, using original TML names for display."""

        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        mark = {} 
        tml_type = chart.get('type', '')
        is_stacked = 'STACKED' in tml_type.upper()

        print(f"\n=== AREA CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        print(f"TML Type: {tml_type}, Is Stacked: {is_stacked}")

        #Extract Custom Axis Names 
        custom_x_axis_title = None
        custom_y_axis_title = None
        try:
            client_state = json.loads(chart.get('client_state_v2', '{}'))
            axis_props = client_state.get('axisProperties', [])
            for prop in axis_props:
                prop_props = prop.get('properties', {})
                if prop_props.get('axisType') == 'X':
                    custom_x_axis_title = prop_props.get('name')
                elif prop_props.get('axisType') == 'Y':
                    custom_y_axis_title = prop_props.get('name')
            if custom_x_axis_title: print(f"  Found custom X-Axis Title: '{custom_x_axis_title}'")
            if custom_y_axis_title: print(f"  Found custom Y-Axis Title: '{custom_y_axis_title}'")
        except Exception as e:
            print(f"  WARNING: Could not parse client_state_v2 for custom axis names. {e}")

        # Get Column Mapping ---
        mapping = self._get_mapping_for_viz(viz_id)
        column_mapping = {}
        if mapping is not None:
            common_mapping_str = mapping.get('common_column_mapping')
            specific_mapping_str = mapping.get('databricks_column_mapping_ToBeFilled')
            mapping_str_to_use = None
            if pd.notna(common_mapping_str) and common_mapping_str and common_mapping_str.strip() != '{}':
                 mapping_str_to_use = common_mapping_str
                 print("  Using common_column_mapping for lookups.")
            elif pd.notna(specific_mapping_str) and specific_mapping_str and specific_mapping_str.strip() != '{}':
                 mapping_str_to_use = specific_mapping_str
                 print("  Using databricks_column_mapping_ToBeFilled for lookups.")
            if mapping_str_to_use:
                try: column_mapping = json.loads(mapping_str_to_use)
                except (json.JSONDecodeError, TypeError) as e: print(f"  WARNING: Failed to parse column mapping JSON: {e}")
        else: print("  WARNING: No mapping data found for this visualization.")

        answer_cols = answer.get('answer_columns', [])
        print(f"Answer columns from TML: {[col.get('name') for col in answer_cols]}")

        #TML Config Extraction
        tml_x_config = axis_config.get('x', [])
        tml_y_config = axis_config.get('y', [])
        tml_color_config = axis_config.get('color', [])

        print(f"TML X Config: {tml_x_config}")
        print(f"TML Y Config: {tml_y_config}")
        print(f"TML Color Config: {tml_color_config}")

        #Process X-axis (Dimension)
        if tml_x_config:
            original_field_x = tml_x_config[0] 
            matching_x_col = next((col for col in answer_cols if col.get('name') == original_field_x or clean_field_name(col.get('name')) == clean_field_name(original_field_x)), None)

            if matching_x_col:
                x_col_name_from_answer = matching_x_col.get('name')
                x_base_field = clean_field_name(x_col_name_from_answer)
                x_sql_alias = column_mapping.get(original_field_x, sanitize_alias_name(x_base_field))
                print(f"X-axis: TML={original_field_x}, FoundInAnswer={x_col_name_from_answer}, MappedSQLAlias={x_sql_alias}")

                encodings['x'] = {
                    "fieldName": x_sql_alias,
                    "displayName": original_field_x, 
                    "scale": {"type": infer_scale_type(x_col_name_from_answer)},
                    "axis": {"title": custom_x_axis_title or original_field_x} 
                }
                available.append('x-axis')
            else:
                print(f"X-axis WARNING: No match in answer_cols for TML field '{original_field_x}'")
                missing.append('x-axis')
        else:
            print(f"X-axis WARNING: No TML config found.")
            missing.append('x-axis')

        #Process Y-axis (One or More Measures)
        measure_fields_processed = [] 
        original_y_names = []         
        y_axis_title = "Value"        

        if tml_y_config:
            print(f"Processing {len(tml_y_config)} Y-axis measure field(s)...")
            for original_field_y in tml_y_config: 
                original_y_names.append(original_field_y)
                matching_y_col = next((col for col in answer_cols if col.get('name') == original_field_y or clean_field_name(col.get('name')) == clean_field_name(original_field_y)), None)

                if matching_y_col:
                    y_col_name_from_answer = matching_y_col.get('name')
                    y_base_field = clean_field_name(y_col_name_from_answer)
                    y_sql_alias = column_mapping.get(original_field_y, sanitize_alias_name(y_base_field))
                    print(f"  - Y-axis measure: TML={original_field_y}, FoundInAnswer={y_col_name_from_answer}, MappedSQLAlias={y_sql_alias}")
                    measure_fields_processed.append({"fieldName": y_sql_alias, "originalName": original_field_y})
                else:
                    print(f"Y-axis WARNING: No match in answer_cols for TML field '{original_field_y}'")
                    missing.append(f'y-axis measure: {original_field_y}')

            #Construct Y-axis Encoding
            if len(measure_fields_processed) == 1:
                measure_info = measure_fields_processed[0]
                y_axis_title = custom_y_axis_title or measure_info["originalName"] 
                encodings['y'] = {
                    "fieldName": measure_info["fieldName"],    
                    "displayName": measure_info["originalName"], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title}          
                }
                available.append('y-axis (single measure)')
                print(f"Y-axis: Single field = {measure_info['fieldName']}")
            elif len(measure_fields_processed) > 1:
                y_axis_title = custom_y_axis_title or ", ".join(original_y_names[:3]) + ("..." if len(original_y_names) > 3 else "") 
                encodings['y'] = {
                    "fields": [{"fieldName": m["fieldName"]} for m in measure_fields_processed], 
                    "scale": {"type": "quantitative"},
                    "axis": {"title": y_axis_title} 
                }
                available.append(f'y-axis ({len(measure_fields_processed)} measures)')
                print(f"Y-axis: Multiple fields = {[m['fieldName'] for m in measure_fields_processed]}")
            else: 
                 if not any(m.startswith('y-axis measure:') for m in missing):
                    missing.append('y-axis (measure)')
        else:
             print(f"Y-axis WARNING: No TML config found.")
             missing.append('y-axis (measure)')

        #Process Color (Dimension for Stacking)
        color_dimension_present = False
        if tml_color_config:
            original_field_color = tml_color_config[0] 
            matching_color_col = next((col for col in answer_cols if col.get('name') == original_field_color or clean_field_name(col.get('name')) == clean_field_name(original_field_color)), None)

            if matching_color_col:
                color_col_name_from_answer = matching_color_col.get('name')
                color_base_field = clean_field_name(color_col_name_from_answer)
                color_sql_alias = column_mapping.get(original_field_color, sanitize_alias_name(color_base_field))
                print(f"Color axis: TML={original_field_color}, FoundInAnswer={color_col_name_from_answer}, MappedSQLAlias={color_sql_alias}")

                encodings['color'] = {
                    "fieldName": color_sql_alias,
                    "displayName": original_field_color, 
                    "scale": {"type": "categorical"}
                }
                color_dimension_present = True
                available.append('color (series)')
            else:
                 print(f"Color axis WARNING: No match in answer_cols for TML field '{original_field_color}'")
                 missing.append(f'color axis: {original_field_color}')
        else:
             if len(measure_fields_processed) > 1:
                 print("Color axis: No TML config, Databricks will use multiple Y-fields for color.")
                 available.append('color (implicit from measures)')
             else:
                 print("Color axis: No color dimension specified in TML.")
        
        # Handle Stacking
        if is_stacked:
             if color_dimension_present or len(measure_fields_processed) > 1:
                 mark = {"stackType": "stacked"}
                 available.append('stacking (area)')
                 print("Mark: Stacking enabled for area chart")

        print(f"Final encodings: {encodings}")
        print(f"Final Mark: {mark}")
        print("="*60 + "\n")

        return {
            "widgetType": "area", # Use 'area' for both AREA and STACKED_AREA
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings,
            "mark": mark 
        }

    def _get_scatter_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate scatter plot specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Scatter plot specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)},
                    "axis": {"title": base_field}
                }
                available.append('x-axis')
            else:
                missing.append('x-axis')
        else:
            missing.append('x-axis')
        
        # Process Y-axis
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"},
                    "axis": {"title": base_field}
                }
                available.append('y-axis')
            else:
                missing.append('y-axis')
        else:
            missing.append('y-axis')
        
        # Process Color (optional)
        if axis_config.get('color') and len(axis_config['color']) > 0:
            original_field = axis_config['color'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color')
        
        # Process Size (optional)
        if axis_config.get('size') and len(axis_config['size']) > 0:
            original_field = axis_config['size'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['size'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('size')
        
        return {
            "widgetType": "scatter",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_table_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate table specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Table specification with widgetType, version, frame, and encodings
        """
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        # Get ordered columns from table config or answer columns
        ordered_cols = answer.get('table', {}).get('ordered_column_ids', [])
        if not ordered_cols:
            ordered_cols = [c.get('name') for c in answer.get('answer_columns', []) if c.get('name')]
        
        answer_cols = answer.get('answer_columns', [])
        columns = []
        
        if ordered_cols:
            available.append('columns')
            for col_name in ordered_cols:
                if not col_name:
                    continue
                
                # Find matching answer column for proper type inference
                matching_col = None
                for col in answer_cols:
                    answer_col_name = col.get('name')
                    if answer_col_name == col_name or clean_field_name(answer_col_name) == clean_field_name(col_name):
                        matching_col = col
                        break
                
                if matching_col:
                    base_field = clean_field_name(matching_col.get('name'))
                    sql_alias = sanitize_alias_name(base_field)
                    
                    # Infer display type from column type
                    col_type = matching_col.get('type', 'string').lower()
                    display_as = "string"
                    if col_type in ['int', 'integer', 'long', 'bigint', 'float', 'double', 'decimal', 'numeric']:
                        display_as = "number"
                    elif col_type in ['date', 'timestamp', 'datetime']:
                        display_as = "datetime"
                    elif col_type in ['boolean', 'bool']:
                        display_as = "boolean"
                    
                    columns.append({
                        "fieldName": sql_alias,
                        "title": base_field,
                        "visible": True,
                        "alignContent": "left",
                        "allowHTML": False,
                        "displayAs": display_as,
                        "type": col_type if col_type else "string"
                    })
                else:
                    # Fallback if column not found in answer_columns
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    columns.append({
                        "fieldName": sql_alias,
                        "title": base_field,
                        "visible": True,
                        "alignContent": "left",
                        "allowHTML": False,
                        "displayAs": "string",
                        "type": "string"
                    })
        else:
            missing.append('columns')
        
        return {
            "widgetType": "table",
            "version": 1,
            "frame": self._get_base_frame(answer),
            "encodings": {"columns": columns},
            "allowHTMLByDefault": False,
            "itemsPerPage": 25
        }

    def _get_counter_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate counter/KPI specification with encodings matching SQL aliases.
        
        When temporal data is present, returns a TUPLE of (counter_spec, line_spec)
        to create both a KPI counter and a trend line chart below it.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict OR tuple: Counter spec, or (counter_spec, line_spec) if temporal data exists
        """
        widget_name = answer.get('name', 'Unknown')
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        # Get answer columns and axis config
        answer_cols = answer.get('answer_columns', [])
        axis_config = chart.get('axis_configs', [{}])[0] if chart.get('axis_configs') else {}
        
        # Check if we have temporal data (x-axis with date/time field)
        has_temporal_field = False
        temporal_field_name = None
        
        # Check x-axis for temporal field
        if axis_config.get('x') and len(axis_config['x']) > 0:
            x_field = axis_config['x'][0]
            
            # Find matching answer column
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == x_field or clean_field_name(col_name) == clean_field_name(x_field):
                    # Check if it's a temporal field (contains Month, Day, Year, etc.)
                    if any(keyword in col_name for keyword in ['Month(', 'Day(', 'Week(', 'Year(', 'Date']):
                        has_temporal_field = True
                        temporal_field_name = col_name
                        print(f"  INFO: Detected temporal field '{temporal_field_name}' for KPI trend visualization")
                    break
        
        # Process primary value field (y-axis or first measure column)
        value_field_alias = None
        value_field_display = None
        
        if axis_config.get('y') and len(axis_config['y']) > 0:
            # Get from y-axis if specified
            y_field = axis_config['y'][0]
            
            # Find matching answer column
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == y_field or clean_field_name(col_name) == clean_field_name(y_field):
                    base_field = clean_field_name(col_name)
                    value_field_alias = sanitize_alias_name(base_field)
                    value_field_display = base_field
                    break
        
        # If no value found from y-axis, find first non-temporal column
        if not value_field_alias and answer_cols:
            for col in answer_cols:
                col_name = col.get('name')
                if col_name:
                    # Skip temporal columns
                    if not any(keyword in col_name for keyword in ['Month(', 'Day(', 'Week(', 'Year(', 'Date']):
                        base_field = clean_field_name(col_name)
                        value_field_alias = sanitize_alias_name(base_field)
                        value_field_display = base_field
                        break
        
        if not value_field_alias:
            missing.append('value')
            return {
                "widgetType": "counter",
                "version": 2,
                "frame": self._get_base_frame(answer),
                "encodings": {}
            }
        
        # BUILD COUNTER SPEC (always simple, no x-axis)
        counter_encodings = {
            "value": {
                "fieldName": value_field_alias,
                "displayName": value_field_display or value_field_alias
            }
        }
        
        counter_spec = {
            "widgetType": "counter",
            "version": 2,
            "frame": self._get_base_frame(answer),
            "encodings": counter_encodings
        }
        
        available.append('value')
        
        # If we have temporal data, CREATE SEPARATE LINE CHART for trend
        if has_temporal_field and temporal_field_name:
            print(f"  INFO: Creating separate line chart for KPI trend using _get_line_spec")
            
            # Create separate lists for line chart tracking
            line_available = []
            line_missing = []
            line_unmapped = []
            
            # Call the existing line chart spec builder
            line_spec = self._get_line_spec(answer, chart, line_available, line_missing, line_unmapped, viz_id)
            
            # Customize the frame to hide title for trend chart
            line_spec['frame']['title'] = f"{answer.get('name', 'Trend')} - Trend"
            line_spec['frame']['showTitle'] = False  # Hide title for cleaner look
            
            # Merge tracking info
            available.extend(line_available)
            missing.extend(line_missing)
            unmapped_props.extend(line_unmapped)
            
            # Return TUPLE: (counter, line chart)
            return (counter_spec, line_spec)
        
        # No temporal data - return simple counter only
        return counter_spec

    def _get_heatmap_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate heatmap specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Heatmap specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)}
                }
                available.append('x')
            else:
                missing.append('x')
        else:
            missing.append('x')
        
        # Process Y-axis
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)}
                }
                available.append('y')
            else:
                missing.append('y')
        else:
            missing.append('y')
        
        # Process Color (intensity) - optional but typical for heatmaps
        if axis_config.get('color') and len(axis_config['color']) > 0:
            original_field = axis_config['color'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('color')
        
        return {
            "widgetType": "heatmap",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_choropleth_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate choropleth (geo map) specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Choropleth specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis (geographic field)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('geographic field')
            else:
                missing.append('geographic field')
        else:
            missing.append('geographic field')
        
        # Process Y-axis (value field)
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('value')
            else:
                missing.append('value')
        else:
            missing.append('value')
        
        return {
            "widgetType": "choropleth map",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_funnel_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate funnel specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Funnel specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis (stage/category field)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('stage')
            else:
                missing.append('stage')
        else:
            missing.append('stage')
        
        # Process Y-axis (value field)
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['y'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "quantitative"}
                }
                available.append('value')
            else:
                missing.append('value')
        else:
            missing.append('value')
        
        return {
            "widgetType": "funnel",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _get_sankey_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate Sankey diagram specification with encodings matching SQL aliases.
        
        Sankey charts require:
        - stages: Array of categorical fields representing the flow nodes
        - value: Quantitative field for flow thickness/weight
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: Unused parameter for consistency
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Sankey specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        print(f"\n=== SANKEY CHART DEBUG for '{widget_name}' (viz_id: {viz_id}) ===")
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
                print(f"Column mapping loaded: {column_mapping}")
            except (json.JSONDecodeError, TypeError) as e:
                print(f"Failed to parse column mapping: {e}")
        
        answer_cols = answer.get('answer_columns', [])
        
        # Collect stage fields (the flow nodes)
        stages = []
        
        # Process X-axis fields (typically source and intermediate nodes)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            for original_field in axis_config['x']:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    print(f"Stage field: original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                    
                    stages.append({
                        "fieldName": sql_alias
                    })
                else:
                    print(f"WARNING: Could not find matching column for stage field {original_field}")
        
        # Process Color axis fields (typically target node)
        if axis_config.get('color') and len(axis_config['color']) > 0:
            for original_field in axis_config['color']:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    print(f"Stage field (from color): original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                    
                    # Avoid duplicates
                    if not any(s['fieldName'] == sql_alias for s in stages):
                        stages.append({
                            "fieldName": sql_alias
                        })
                else:
                    print(f"WARNING: Could not find matching column for stage field {original_field}")
        
        if stages:
            encodings['stages'] = stages
            available.append(f'stages ({len(stages)})')
            print(f"Added {len(stages)} stage fields")
        else:
            missing.append('stages')
            print("WARNING: No stage fields found for Sankey")
        
        # Process Y-axis (value field) - the measure for flow weight
        if axis_config.get('y') and len(axis_config['y']) > 0:
            original_field = axis_config['y'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                print(f"Value field: original={original_field}, col_name={col_name}, base={base_field}, sql_alias={sql_alias}")
                
                encodings['value'] = {
                    "fieldName": sql_alias
                }
                available.append('value')
            else:
                print(f"WARNING: Could not find matching column for value field {original_field}")
                missing.append('value')
        else:
            missing.append('value')
        
        print(f"Final Sankey encodings: stages={[s['fieldName'] for s in encodings.get('stages', [])]}, value={encodings.get('value', {}).get('fieldName')}")
        print("="*50 + "\n")
        
        return {
            "widgetType": "sankey",
            "version": 1,  # Changed to version 1 to match your example
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }


    def _get_pivot_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate pivot table specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: List to track optional unmapped properties
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Pivot table specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        # Get ordered columns
        ordered_cols = answer.get('table', {}).get('ordered_column_ids', [])
        if not ordered_cols:
            ordered_cols = [c.get('name') for c in answer.get('answer_columns', []) if c.get('name')]
        
        answer_cols = answer.get('answer_columns', [])
        axis_config = chart.get('axis_configs', [{}])[0] if chart.get('axis_configs') else {}
        
        encodings = {}
        
        # Process Rows (x-axis)
        if axis_config.get('x') and len(axis_config['x']) > 0:
            rows = []
            for original_field in axis_config['x']:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    rows.append({"fieldName": sql_alias, "displayName": base_field})
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    rows.append({"fieldName": sql_alias, "displayName": base_field})
            
            encodings['rows'] = rows
            available.append('rows')
        elif ordered_cols:
            original_field = ordered_cols[0]
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                encodings['rows'] = [{"fieldName": sql_alias, "displayName": base_field}]
            else:
                base_field = clean_field_name(original_field)
                sql_alias = sanitize_alias_name(base_field)
                encodings['rows'] = [{"fieldName": sql_alias, "displayName": base_field}]
            available.append('rows (inferred)')
        else:
            missing.append('rows')
        
        # Process Columns (color axis) - optional for pivot
        if axis_config.get('color') and len(axis_config['color']) > 0:
            columns = []
            for original_field in axis_config['color']:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    columns.append({"fieldName": sql_alias, "displayName": base_field})
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    columns.append({"fieldName": sql_alias, "displayName": base_field})
            
            encodings['columns'] = columns
            available.append('columns')
        else:
            unmapped_props.append('columns (pivot)')
        
        # Process Values (y-axis) - Convert to cell structure for pivot
        if axis_config.get('y') and len(axis_config['y']) > 0:
            fields = []
            for original_field in axis_config['y']:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    # Use the field name directly without wrapping in aggregation
                    fields.append({
                        "fieldName": sql_alias,  # 
                        "cellType": "text"
                    })
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias,  
                        "cellType": "text"
                    })
            
            encodings['cell'] = {
                "type": "multi-cell",
                "fields": fields
            }
            available.append('cell')
        elif len(ordered_cols) > 1:
            fields = []
            for original_field in ordered_cols[1:]:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias,  # ✅ Just use sql_alias directly
                        "cellType": "text"
                    })
                else:
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    
                    fields.append({
                        "fieldName": sql_alias, 
                        "cellType": "text"
                    })
            
            encodings['cell'] = {
                "type": "multi-cell",
                "fields": fields
            }
            available.append('cell (inferred)')
        else:
            missing.append('cell')
        return {
        "widgetType": "pivot",
        "version": 3,
        "frame": self._get_base_frame(answer),
        "encodings": encodings
        }
    def _get_combo_spec(self, answer, chart, available, missing, unmapped_props, viz_id=None):
        """
        Generate combo chart specification with encodings matching SQL aliases.
        
        Args:
            answer: Answer object containing chart data and columns
            chart: Chart configuration with axis settings
            available: List to track successfully mapped encodings
            missing: List to track missing/failed encodings
            unmapped_props: List to track optional unmapped properties
            viz_id: Optional visualization ID for column mapping lookup
            
        Returns:
            dict: Combo chart specification with widgetType, version, frame, and encodings
        """
        widget_name = answer.get('name', 'Unknown')
        axis_config = chart.get('axis_configs', [{}])[0]
        encodings = {}
        
        # Load column mapping if available
        column_mapping = {}
        mapping = self._get_mapping_for_viz(viz_id) if viz_id else None
        
        if mapping is not None and mapping.get('databricks_column_mapping_ToBeFilled'):
            try:
                column_mapping = json.loads(mapping['databricks_column_mapping_ToBeFilled'])
            except (json.JSONDecodeError, TypeError):
                pass  # Continue with empty mapping
        
        answer_cols = answer.get('answer_columns', [])
        
        # Process X-axis
        if axis_config.get('x') and len(axis_config['x']) > 0:
            original_field = axis_config['x'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['x'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": infer_scale_type(base_field)},
                    "axis": {"title": base_field}
                }
                available.append('x-axis')
            else:
                missing.append('x-axis')
        else:
            missing.append('x-axis')
        
        # Process Y-axis (can contain multiple fields for combo charts)
        if axis_config.get('y') and len(axis_config['y']) > 0:
            y_fields = axis_config['y']
            transformed_fields = []
            
            for original_field in y_fields:
                # Find matching answer column
                matching_col = None
                for col in answer_cols:
                    col_name = col.get('name')
                    if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                        matching_col = col
                        break
                
                if matching_col:
                    col_name = matching_col.get('name')
                    base_field = clean_field_name(col_name)
                    sql_alias = sanitize_alias_name(base_field)
                    transformed_fields.append(sql_alias)
                else:
                    # Fallback if no match found
                    base_field = clean_field_name(original_field)
                    sql_alias = sanitize_alias_name(base_field)
                    transformed_fields.append(sql_alias)
            
            encodings['y'] = {
                "fieldName": transformed_fields,
                "displayName": "Value",
                "scale": {"type": "quantitative"},
                "axis": {"title": "Value"}
            }
            available.append('y-axis')
        else:
            missing.append('y-axis')
        
        # Process Color (series) - optional
        if axis_config.get('color') and len(axis_config['color']) > 0:
            original_field = axis_config['color'][0]
            
            # Find matching answer column
            matching_col = None
            for col in answer_cols:
                col_name = col.get('name')
                if col_name == original_field or clean_field_name(col_name) == clean_field_name(original_field):
                    matching_col = col
                    break
            
            if matching_col:
                col_name = matching_col.get('name')
                base_field = clean_field_name(col_name)
                sql_alias = sanitize_alias_name(base_field)
                
                encodings['color'] = {
                    "fieldName": sql_alias,
                    "displayName": base_field,
                    "scale": {"type": "categorical"}
                }
                available.append('color (series)')
        
        unmapped_props.append('series chart type mapping (combo charts partially supported)')
        
        return {
            "widgetType": "combo",
            "version": 3,
            "frame": self._get_base_frame(answer),
            "encodings": encodings
        }

    def _create_layout(self, layout_data, widgets):
        """
        Create dashboard layout with widget positions using database mappings.
        Handles trend charts by placing them directly below their parent counters.
        """
        layout = []
        GRID_WIDTH = 6
        
        # Separate regular widgets from trend charts
        regular_widgets = [w for w in widgets if not w.get('is_trend_chart')]
        trend_widgets = {w.get('parent_viz_id'): w for w in widgets if w.get('is_trend_chart')}
        
        tiles = layout_data.get('tiles', []) if layout_data else []
        
        if tiles:
            # Use TML tile positions
            x = 0
            y = 0
            row_height = 0
            
            for tile in tiles:
                viz_id = tile.get('visualization_id')
                widget_obj = next((w for w in regular_widgets if w.get('viz_id') == viz_id), None)
                
                if not widget_obj:
                    print(f"WARNING: Widget not found for viz_id {viz_id}, skipping tile")
                    continue
                
                if not widget_obj.get('widget'):
                    print(f"WARNING: Widget object missing 'widget' key for viz_id {viz_id}")
                    continue
                
                # Get size from tile or default
                tile_size = tile.get('size')
                if not tile_size:
                    widget_type = widget_obj.get('widget', {}).get('spec', {}).get('widgetType')
                    tile_size = self._get_default_size_for_widget_type(widget_type)
                
                size = self._get_widget_size_from_db(tile_size)
                if not size:
                    print(f"WARNING: No size configuration found for widget {viz_id}. Skipping.")
                    continue
                
                # Handle grid wrapping
                if x + size['width'] > GRID_WIDTH:
                    x = 0
                    y += row_height
                    row_height = 0
                
                # Add main widget
                layout.append({
                    "widget": widget_obj['widget'],
                    "position": {
                        "x": x, 
                        "y": y, 
                        "width": size['width'], 
                        "height": size['height']
                    }
                })
                
                current_x = x
                current_y = y
                current_height = size['height']
                
                # Check if this widget has a trend chart
                if viz_id in trend_widgets:
                    trend_widget = trend_widgets[viz_id]
                    
                    # Get size for trend chart (use LARGE for better visibility)
                    trend_size = self._get_widget_size_from_db('LARGE')
                    if not trend_size:
                        print(f"WARNING: Could not get size for trend chart of {viz_id}")
                    else:
                        # Place trend chart directly below the counter
                        trend_y = current_y + current_height
                        
                        layout.append({
                            "widget": trend_widget['widget'],
                            "position": {
                                "x": current_x,  # Same X position as counter
                                "y": trend_y,     # Below the counter
                                "width": trend_size['width'], 
                                "height": trend_size['height']
                            }
                        })
                        
                        print(f"  ✓ Added trend chart for {viz_id} at position (x={current_x}, y={trend_y})")
                        
                        # Update row height to account for trend chart
                        row_height = max(row_height, current_height + trend_size['height'])
                
                x += size['width']
                row_height = max(row_height, current_height)
        
        else:
            # No tiles - use default layout
            x = 0
            y = 0
            
            for widget_obj in regular_widgets:
                if not widget_obj or not widget_obj.get('widget'):
                    continue
                
                widget_type = widget_obj.get('widget', {}).get('spec', {}).get('widgetType')
                if not widget_type:
                    continue
                
                default_size_category = self._get_default_size_for_widget_type(widget_type)
                size = self._get_widget_size_from_db(default_size_category)
                if not size:
                    continue
                
                # Handle grid wrapping
                if x + size['width'] > GRID_WIDTH:
                    x = 0
                    y += size['height']
                
                # Add main widget
                layout.append({
                    "widget": widget_obj['widget'],
                    "position": {
                        "x": x, 
                        "y": y, 
                        "width": size['width'], 
                        "height": size['height']
                    }
                })
                
                current_x = x
                current_y = y
                current_height = size['height']
                
                # Check if this widget has a trend chart
                viz_id = widget_obj.get('viz_id')
                if viz_id in trend_widgets:
                    trend_widget = trend_widgets[viz_id]
                    trend_size = self._get_widget_size_from_db('LARGE')
                    
                    if trend_size:
                        trend_y = current_y + current_height
                        
                        layout.append({
                            "widget": trend_widget['widget'],
                            "position": {
                                "x": current_x,
                                "y": trend_y,
                                "width": trend_size['width'],
                                "height": trend_size['height']
                            }
                        })
                        
                        print(f"  ✓ Added trend chart for {viz_id} at position (x={current_x}, y={trend_y})")
                
                x += size['width']
        
        return layout


    def _get_widget_size_from_db(self, size_category):
        """
        Fetch widget dimensions from pre-loaded WIDGET_SIZE_MAP.
        
        Args:
            size_category: Size category (e.g., 'SMALL', 'MEDIUM', 'LARGE')
            
        Returns:
            dict: Dictionary with 'width' and 'height' keys, or None if not found
        """
        if not size_category:
            print(f"WARNING: _get_widget_size_from_db called with no size category")
            return None
        
        # Use the pre-loaded WIDGET_SIZE_MAP from global scope
        size_config = WIDGET_SIZE_MAP.get(size_category)
        
        if size_config:
            print(f"INFO: Found size config for '{size_category}': width={size_config['width']}, height={size_config['height']}")
            return size_config
        
        print(f"WARNING: No size configuration found for category '{size_category}' in WIDGET_SIZE_MAP")
        print(f"Available size categories: {list(WIDGET_SIZE_MAP.keys())}")
        return None


    def _get_default_size_for_widget_type(self, widget_type):
        """
        Fetch default size category for a widget type from pre-loaded TML_TO_LVDASH_MAPPING.
        
        Args:
            widget_type: Widget type (e.g., 'bar', 'table', 'counter')
            
        Returns:
            str: Size category (e.g., 'MEDIUM'), or None if not found
        """
        normalized_widget_type = widget_type.replace(' ', '_')
        if not widget_type:
            print(f"WARNING: _get_default_size_for_widget_type called with no widget type")
            return None
        
        # Query the chart_type_mappings table for default_size
        try:
            from pyspark.sql import SparkSession
            spark = SparkSession.builder.getOrCreate()
            
            result = spark.sql(f"""
                SELECT default_size 
                FROM {CHART_TYPE_MAPPING_TABLE}
                WHERE widget_type = '{normalized_widget_type}'
                LIMIT 1
            """).collect()
            
            if result and len(result) > 0:
                default_size = result[0]['default_size']
                if default_size:
                    print(f"INFO: Found default size '{default_size}' for widget type '{normalized_widget_type}'")
                    return default_size
            
            print(f"WARNING: No default size found for widget type '{normalized_widget_type}'")
            print(f"Hint: Check if '{normalized_widget_type}' exists in {CHART_TYPE_MAPPING_TABLE}")
            return None
            
        except Exception as e:
            print(f"ERROR: Failed to fetch default size for widget type '{normalized_widget_type}': {e}")
            import traceback
            traceback.print_exc()
            return None

def convert_all_tml_files():
    setup_environment()
    
    # ===== NEW: Load liveboard configuration =====
    CONFIG_TABLE = f"{CATALOG}.{SCHEMA}.liveboard_migration_config"
    
    print("\n--- Loading Liveboard Configuration ---")
    try:
        config_df = spark.table(CONFIG_TABLE).toPandas()
        enabled_liveboards = config_df[config_df['process_flag'] == 'Y']
        
        if len(enabled_liveboards) == 0:
            print(f"WARNING: No liveboards enabled (process_flag='Y') in {CONFIG_TABLE}")
            print("Please enable at least one liveboard:")
            print(f"UPDATE {CONFIG_TABLE} SET process_flag = 'Y' WHERE name = 'Your Dashboard Name';")
            return
        
        enabled_names = set(enabled_liveboards['name'].tolist())
        enabled_guids = set(enabled_liveboards['guid'].tolist())
        
        print(f"Found {len(enabled_liveboards)} enabled liveboards:")
        for _, row in enabled_liveboards.iterrows():
            print(f"  ✓ {row['name']} ({row['guid']})")
        print("---\n")
        
    except Exception as e:
        print(f"ERROR: Could not load configuration table {CONFIG_TABLE}: {e}")
        print("Proceeding to convert ALL liveboards without filtering...")
        enabled_names = None
        enabled_guids = None
    # ===== END NEW SECTION =====
    
    if MAPPING_DATA is not None and len(MAPPING_DATA) > 0:
        total_mappings = len(MAPPING_DATA)
        mapped_count = len(MAPPING_DATA[
            (MAPPING_DATA['databricks_table_name_ToBeFilled'].notna()) & 
            (MAPPING_DATA['databricks_table_name_ToBeFilled'] != '')
        ])
        print(f"\n--- Mapping Status ---")
        print(f"Total visualizations: {total_mappings}")
        print(f"Mapped to Databricks tables: {mapped_count}")
        print(f"Unmapped (will use TML table names): {total_mappings - mapped_count}")
        print("---\n")
    
    try:
        tml_files = [f.path for f in dbutils.fs.ls(TML_INPUT_PATH) if f.path.endswith(('.tml', '.yaml', '.json'))]
    except Exception as e:
        print(f"ERROR: Cannot list files in '{TML_INPUT_PATH}'. Error: {e}")
        return

    if not tml_files:
        print(f"No TML files found in {TML_INPUT_PATH}")
        return
    
    # ===== NEW: Filter TML files based on enabled liveboards =====
    if enabled_names or enabled_guids:
        filtered_files = []
        skipped_files = []
        
        for file_path in tml_files:
            filename = Path(file_path).name
            
            try:
                tml_data = parse_tml_file(file_path)
                liveboard = tml_data.get('liveboard', {})
                lb_name = liveboard.get('name', '')
                lb_guid = tml_data.get('guid', '')
                
                if lb_name in enabled_names or lb_guid in enabled_guids:
                    filtered_files.append(file_path)
                    print(f"  ✓ Including: {filename} ({lb_name})")
                else:
                    skipped_files.append(filename)
                    
            except Exception as e:
                print(f"  WARNING: Could not parse {filename} for filtering: {e}")
                filtered_files.append(file_path)
        
        tml_files = filtered_files
        
        print(f"\n--- File Filtering Results ---")
        print(f"Total files found: {len(tml_files) + len(skipped_files)}")
        print(f"Files to process: {len(tml_files)}")
        print(f"Files skipped: {len(skipped_files)}")
        if skipped_files and len(skipped_files) <= 10:
            print(f"Skipped files: {', '.join(skipped_files)}")
        elif len(skipped_files) > 10:
            print(f"Skipped files: {', '.join(skipped_files[:10])}... and {len(skipped_files)-10} more")
        print("---\n")
    
    if not tml_files:
        print("No enabled TML files to process after filtering.")
        return
    
    print(f"Processing {len(tml_files)} enabled liveboard(s)...\n")
    # ===== END NEW SECTION =====
    
    tracker = ConversionTracker()
    summary_results = []
    failure_records = []
    
    for tml_file_path in tml_files:
        filename = Path(tml_file_path).name
        status = 'ERROR'
        lvdash_data = {}
        output_filename = 'N/A'
        
        try:
            print(f"\n--- Processing: {filename} ---")
            
            try:
                tml_data = parse_tml_file(tml_file_path)
            except Exception as parse_error:
                raise ValueError(f"Failed to parse TML file: {str(parse_error)}")
            
            converter = TMLToLVDASHConverter(tml_data, filename, tracker, MAPPING_DATA)
            lvdash_data = converter.convert()
            
            output_filename = filename.replace('.tml', '').replace('.yaml', '').replace('.json', '') + '.lvdash.json'
            output_path = os.path.join(LVDASH_OUTPUT_PATH, output_filename)
            dbutils.fs.put(output_path, json.dumps(lvdash_data, indent=2), overwrite=True)
            status = 'SUCCESS'
            print(f"  Saved to {output_filename}")

        except Exception as e:
            print(f"  FAILED: {e}")
            import traceback
            error_trace = traceback.format_exc()
            traceback.print_exc()
            
            failure_records.append({
                'tml_file': filename,
                'error_type': type(e).__name__,
                'error_message': str(e)[:1000],
                'stack_trace': error_trace[:2000],
                'failure_timestamp': datetime.now()
            })
        
        summary_results.append({
            'tml_file': filename, 
            'lvdash_file': output_filename, 
            'status': status,
            'num_datasets': len(lvdash_data.get('datasets', [])),
            'num_pages': len(lvdash_data.get('pages', [])),
            'num_widgets': sum(len(p.get('layout', [])) for p in lvdash_data.get('pages', [])),
            'conversion_timestamp': datetime.now()
        })

    print("\n--- Saving logs ---")
    if tracker.records:
        df = pd.DataFrame(tracker.records)
        df['conversion_timestamp'] = pd.to_datetime(df['conversion_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(TRACKER_TABLE)
        print(f"Saved {len(tracker.records)} widget records to {TRACKER_TABLE}")

    if summary_results:
        df = pd.DataFrame(summary_results)
        df['num_datasets'] = df['num_datasets'].astype('int32')
        df['num_pages'] = df['num_pages'].astype('int32')
        df['num_widgets'] = df['num_widgets'].astype('int32')
        df['conversion_timestamp'] = pd.to_datetime(df['conversion_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(SUMMARY_TABLE)
        print(f"Saved {len(summary_results)} file summaries to {SUMMARY_TABLE}")
    
    if failure_records:
        df = pd.DataFrame(failure_records)
        df['failure_timestamp'] = pd.to_datetime(df['failure_timestamp'])
        spark_df = spark.createDataFrame(df)
        spark_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(FAILURE_TABLE)
        print(f"Saved {len(failure_records)} failures to {FAILURE_TABLE}")
    
    print("\n--- Conversion complete! ---")

convert_all_tml_files()

print("--- Overall Conversion Summary ---")
try:
    display(spark.table(SUMMARY_TABLE).orderBy("conversion_timestamp", ascending=False))
except Exception as e:
    print(f"Could not display summary table. Error: {e}")

print("\n--- Detailed Widget Conversion Tracker ---")
try:
    display(spark.table(TRACKER_TABLE).orderBy("conversion_timestamp", ascending=False))
except Exception as e:
    print(f"Could not display tracker table. Error: {e}")

print("\n--- Conversion Failures ---")
try:
    fail_df = spark.table(FAILURE_TABLE)
    fail_count = fail_df.count()
    if fail_count > 0:
        print(f"Found {fail_count} failed conversions:")
        display(fail_df.orderBy("failure_timestamp", ascending=False))
    else:
        print("No failures!")
except Exception as e:
    print(f"Could not display failure table. Error: {e}")

print("\n--- Chart Type Mapping Statistics ---")
try:
    stats_query = f"""
    SELECT 
        tml_type,
        lvdash_type,
        status,
        COUNT(*) as count,
        COUNT(CASE WHEN unmapped_properties != '' THEN 1 END) as has_unmapped_props
    FROM {TRACKER_TABLE}
    GROUP BY tml_type, lvdash_type, status
    ORDER BY count DESC
    """
    display(spark.sql(stats_query))
except Exception as e:
    print(f"Could not display statistics. Error: {e}")

print("\n--- Unmapped Properties Report ---")
try:
    unmapped_query = f"""
    SELECT 
        tml_file,
        widget_name,
        tml_type,
        lvdash_type,
        unmapped_properties,
        notes
    FROM {TRACKER_TABLE}
    WHERE unmapped_properties != '' OR notes != ''
    ORDER BY tml_file, widget_name
    """
    display(spark.sql(unmapped_query))
except Exception as e:
    print(f"Could not display unmapped properties. Error: {e}")